In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from google.colab import drive

%matplotlib inline
drive.mount("/drive")

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/drive/MyDrive/Dataset.csv")
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")

In [ ]:
# Calculate MAPE for each C&FAgent.

df_2006 = df[(df["Date"].dt.year == 2006) & (df["Sales"] != 0)].copy()
df_2006["MAPE"] = abs(df_2006["Sales"] - df_2006["DemandForecast"]) / df_2006["Sales"] * 100
df_mape = df_2006.groupby("C&FAgent").agg({"MAPE": "mean"}).reset_index()
df_mape

,C&FAgent,MAPE
0,Alabama Agent1,101.475956
1,Alabama Agent2,116.051129
2,Alabama Agent3,112.820729
3,California Agent1,97.541271
4,California Agent2,154.234268
5,Colorado Agent1,88.688754
6,Colorado Agent2,141.007858
7,Colorado Agent3,121.418387
8,Colorado Agent4,118.833510
9,Colorado Agent5,147.328159


In [ ]:
# Sort DataFrame by MAPE values in descending order
df_mape = df.sort_values(by='MAPE', ascending=False)

print(df_mape)

             C&FAgent        MAPE
21    Michigan Agent1  309.481804
24    Michigan Agent4  293.223756
16     Indiana Agent1  186.567176
26    Michigan Agent6  185.676224
20   Louisiana Agent2  178.100444
4   California Agent2  154.234268
13      Hawaii Agent1  149.550938
9     Colorado Agent5  147.328159
6     Colorado Agent2  141.007858
23    Michigan Agent3  137.776062
10     Florida Agent1  135.269539
28       Texas Agent2  131.936814
19   Louisiana Agent1  121.969621
7     Colorado Agent3  121.418387
8     Colorado Agent4  118.833510
1      Alabama Agent2  116.051129
2      Alabama Agent3  112.820729
27       Texas Agent1  107.167023
12     Florida Agent3  105.861623
15      Hawaii Agent3  105.693099
11     Florida Agent2  102.703490
0      Alabama Agent1  101.475956
14      Hawaii Agent2  101.212263
18    Kentucky Agent1   98.463066
3   California Agent1   97.541271
22    Michigan Agent2   93.133450
17     Indiana Agent2   89.650944
5     Colorado Agent1   88.688754
25    Michigan

In [ ]:
df_pivot = pd.pivot_table(df, values="Sales", index=["C&FAgent", "ProductCode"], aggfunc='sum').reset_index()
df_pivot

,C&FAgent,ProductCode,Sales
0,Alabama Agent1,15645,90275
1,Alabama Agent1,16220,69495
2,Alabama Agent1,17240,88218
3,Alabama Agent1,19875,24919
4,Alabama Agent1,19920,111250
...,...,...,...
812,Texas Agent2,95945,278
813,Texas Agent2,96500,17214
814,Texas Agent2,97533,199
815,Texas Agent2,98669,165


In [ ]:
df_total_sales = df.groupby("C&FAgent").agg({"Sales": "sum"}).reset_index()
df_total_sales

,C&FAgent,Sales
0,Alabama Agent1,60940838
1,Alabama Agent2,46147054
2,Alabama Agent3,19406093
3,California Agent1,35365379
4,California Agent2,4656928
5,Colorado Agent1,39339360
6,Colorado Agent2,19543433
7,Colorado Agent3,8416713
8,Colorado Agent4,21566941
9,Colorado Agent5,10790985


In [ ]:
df_pctage = pd.merge(df_pivot, df_total_sales, on="C&FAgent").rename(columns={"Sales_x": "Sales", "Sales_y": "C&FAgent_Sales"})
df_pctage["Percentage"] = round(df_pctage["Sales"] / df_pctage["C&FAgent_Sales"] * 100, 2)
df_pctage

,C&FAgent,ProductCode,Sales,C&FAgent_Sales,Percentage
0,Alabama Agent1,15645,90275,60940838,0.15
1,Alabama Agent1,16220,69495,60940838,0.11
2,Alabama Agent1,17240,88218,60940838,0.14
3,Alabama Agent1,19875,24919,60940838,0.04
4,Alabama Agent1,19920,111250,60940838,0.18
...,...,...,...,...,...
812,Texas Agent2,95945,278,79606628,0.00
813,Texas Agent2,96500,17214,79606628,0.02
814,Texas Agent2,97533,199,79606628,0.00
815,Texas Agent2,98669,165,79606628,0.00
